# Import Libraries

In [1]:
import os
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
from read_data import combine_all, combine_category, read_data

# Read Dataset

In [5]:
df1 = pd.read_csv('mr_beast/0e3GPea1Tyg.csv')
df2 = pd.read_csv('mr_beast/9bqk6ZUsKyA.csv')

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190390 entries, 0 to 190389
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Name         190380 non-null  object
 1   Comment      190196 non-null  object
 2   Time         190390 non-null  object
 3   Likes        190390 non-null  int64 
 4   Reply Count  190390 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 7.3+ MB


In [11]:
pd.concat([df1, df2]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712560 entries, 0 to 190389
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Name         712506 non-null  object
 1   Comment      712316 non-null  object
 2   Time         712560 non-null  object
 3   Likes        712560 non-null  int64 
 4   Reply Count  712560 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 32.6+ MB
